<a href="https://www.kaggle.com/code/jackren000/lstm-predict-energy-behavior-of-prosumers?scriptVersionId=159802742" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#### load the libraries
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Step 1: Try raw data

### Data Collection

In [2]:
#### update data directory path
DATA_DIR = '/kaggle/input/predict-energy-behavior-of-prosumers'

In [3]:
#### read the CSV files into DataFrames
train = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))

### Data Exploration

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   county              int64  
 1   is_business         int64  
 2   product_type        int64  
 3   target              float64
 4   is_consumption      int64  
 5   datetime            object 
 6   data_block_id       int64  
 7   row_id              int64  
 8   prediction_unit_id  int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 138.6+ MB


In [5]:
train.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2


In [6]:
# display datetime range
train['datetime'].unique()

array(['2021-09-01 00:00:00', '2021-09-01 01:00:00',
       '2021-09-01 02:00:00', ..., '2023-05-31 21:00:00',
       '2023-05-31 22:00:00', '2023-05-31 23:00:00'], dtype=object)

Note that in the `train.csv` dataset, the datetime change begins with the hour, followed by the day, and then the month.

Here is the pseudocode of `train.csv` dataset:

In [7]:

################## The pseudocode of the train dataset ##################
#for year in range(2021, 2024):  
#    for month in range(1, 13):  # Adjusted to correctly range from 1 to 12  
#        for hour in range(24):  
#           for county in range(15):  
#                for is_business in range(2):  # Adjusted to correctly range from 0 to 1  
#                    for product in range(4):  
#                        print(target)  
######################################################################

### Data Transformation

In [8]:
train['datetime'] = pd.to_datetime(train['datetime'])

In [9]:
# 'datetime' column is changed to datetime64[ns]
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 9 columns):
 #   Column              Dtype         
---  ------              -----         
 0   county              int64         
 1   is_business         int64         
 2   product_type        int64         
 3   target              float64       
 4   is_consumption      int64         
 5   datetime            datetime64[ns]
 6   data_block_id       int64         
 7   row_id              int64         
 8   prediction_unit_id  int64         
dtypes: datetime64[ns](1), float64(1), int64(7)
memory usage: 138.6 MB


In [10]:
train.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01,0,0,0
1,0,0,1,96.590,1,2021-09-01,0,1,0
2,0,0,2,0.000,0,2021-09-01,0,2,1
3,0,0,2,17.314,1,2021-09-01,0,3,1
4,0,0,3,2.904,0,2021-09-01,0,4,2


In [11]:
# display datetime range
train['datetime'].unique()

<DatetimeArray>
['2021-09-01 00:00:00', '2021-09-01 01:00:00', '2021-09-01 02:00:00',
 '2021-09-01 03:00:00', '2021-09-01 04:00:00', '2021-09-01 05:00:00',
 '2021-09-01 06:00:00', '2021-09-01 07:00:00', '2021-09-01 08:00:00',
 '2021-09-01 09:00:00',
 ...
 '2023-05-31 14:00:00', '2023-05-31 15:00:00', '2023-05-31 16:00:00',
 '2023-05-31 17:00:00', '2023-05-31 18:00:00', '2023-05-31 19:00:00',
 '2023-05-31 20:00:00', '2023-05-31 21:00:00', '2023-05-31 22:00:00',
 '2023-05-31 23:00:00']
Length: 15312, dtype: datetime64[ns]

####################################################

In [12]:
#### read the CSV files into DataFrames
train = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
gas_df = pd.read_csv(os.path.join(DATA_DIR, "gas_prices.csv"))
electricity_df = pd.read_csv(os.path.join(DATA_DIR, "electricity_prices.csv"))
client_df = pd.read_csv(os.path.join(DATA_DIR, "client.csv"))
fw_df = pd.read_csv(os.path.join(DATA_DIR, "forecast_weather.csv"))
hw_df = pd.read_csv(os.path.join(DATA_DIR, "historical_weather.csv"))

# read a file from a different directory
# see how this data is generated: https://www.kaggle.com/jackren000/mapping-locations-and-county-code/edit
# locations = pd.read_csv("/kaggle/input/county-lon-lats/county_lon_lats.csv")

In [13]:
train.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2


In [14]:
################## client
# reduce 'block_id' in the 'client' DataFrame by 2 to match 'train' DataFrame
client_df['data_block_id'] -= 2
client_df.head()

,product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
0,1,0,108,952.89,0,2021-09-01,0
1,2,0,17,166.40,0,2021-09-01,0
2,3,0,688,7207.88,0,2021-09-01,0
3,0,0,5,400.00,1,2021-09-01,0
4,1,0,43,1411.00,1,2021-09-01,0


In [15]:
client_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41919 entries, 0 to 41918
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_type        41919 non-null  int64  
 1   county              41919 non-null  int64  
 2   eic_count           41919 non-null  int64  
 3   installed_capacity  41919 non-null  float64
 4   is_business         41919 non-null  int64  
 5   date                41919 non-null  object 
 6   data_block_id       41919 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 2.2+ MB


In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   county              int64  
 1   is_business         int64  
 2   product_type        int64  
 3   target              float64
 4   is_consumption      int64  
 5   datetime            object 
 6   data_block_id       int64  
 7   row_id              int64  
 8   prediction_unit_id  int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 138.6+ MB


In [17]:
train = train.merge(client_df[['installed_capacity', 'eic_count', 'data_block_id', 'county', 'is_business', 'product_type']], how='left', on=['data_block_id', 'county', 'is_business', 'product_type'])

In [18]:
train['installed_capacity'].fillna(value=0, inplace=True)
train['eic_count'].fillna(value=0, inplace=True)

In [19]:
# Initialize the scaler.
scaler = MinMaxScaler(feature_range=(0, 1))

train['installed_capacity'] = scaler.fit_transform(train[['installed_capacity']])
train['eic_count'] = scaler.fit_transform(train[['eic_count']])

it has to do with month and hour, not year related.

In [20]:
train['datetime'] = pd.to_datetime(train['datetime'])

# # add year column in train DataFrame by extracting the year part from datetime object
# train['year'] = train['datetime'].dt.year

# # add month column in train DataFrame
train['month'] = train['datetime'].dt.month

# add hour column in train DataFrame
train['hour'] = train['datetime'].dt.hour

# add day of week column in train DataFrame
train['dayofweek'] = train['datetime'].dt.dayofweek

# add day of year column in train DataFrame
# train['dayofyear'] = train['datetime'].dt.dayofyear

In [21]:
# set 'datetime' column as index
train.set_index('datetime', inplace=True)
# sort the index as ascending order
train.sort_index(inplace=True)
# fills any missing values in the 'target' column with 0
train['target'].fillna(value=0, inplace=True)
# train.dropna(axis=0, inplace=True)

# scale the target column
# scale is crucial for the LSTM model
scaler = MinMaxScaler(feature_range=(0, 1))
train['target'] = scaler.fit_transform(train[['target']])

train.head()

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,installed_capacity,eic_count,month,hour,dayofweek
datetime,,,,,,,,,,,,,
2021-09-01,0,0,1,0.000046,0,0,0,0,0.049336,0.071193,9,0,2
2021-09-01,0,0,1,0.006240,1,0,1,0,0.049336,0.071193,9,0,2
2021-09-01,0,0,2,0.000000,0,0,2,1,0.008615,0.011206,9,0,2
2021-09-01,0,0,2,0.001118,1,0,3,1,0.008615,0.011206,9,0,2
2021-09-01,0,0,3,0.000188,0,0,4,2,0.373189,0.453527,9,0,2


In [22]:
# drop non-feature columns from the dataset
X = train.drop(['target', 'row_id', 'data_block_id'], axis=1)
# isolate the target variable
y = train['target']

train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2018352 entries, 2021-09-01 00:00:00 to 2023-05-31 23:00:00
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   county              int64  
 1   is_business         int64  
 2   product_type        int64  
 3   target              float64
 4   is_consumption      int64  
 5   data_block_id       int64  
 6   row_id              int64  
 7   prediction_unit_id  int64  
 8   installed_capacity  float64
 9   eic_count           float64
 10  month               int32  
 11  hour                int32  
 12  dayofweek           int32  
dtypes: float64(3), int32(3), int64(7)
memory usage: 192.5 MB


In [23]:
# split the dataset into 70% training and 30% testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# reshape features for LSTM: [samples, timesteps, features]
X_train = np.array(X_train).reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = np.array(X_test).reshape((X_test.shape[0], 1, X_test.shape[1]))

# # initialize the scaler for the feature columns
# scaler_features = MinMaxScaler(feature_range=(0, 1))
# # fit the scaler on the training feature data and transform it
# X_train_scaled = scaler_features.fit_transform(X_train)
# # transform the test feature data with the same scaler
# X_test_scaled = scaler_features.transform(X_test)

# print the shapes of the train and test data
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (1412846, 1, 10)
y_train shape: (1412846,)
X_test shape: (605506, 1, 10)
y_test shape: (605506,)


In [24]:
train.head()

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,installed_capacity,eic_count,month,hour,dayofweek
datetime,,,,,,,,,,,,,
2021-09-01,0,0,1,0.000046,0,0,0,0,0.049336,0.071193,9,0,2
2021-09-01,0,0,1,0.006240,1,0,1,0,0.049336,0.071193,9,0,2
2021-09-01,0,0,2,0.000000,0,0,2,1,0.008615,0.011206,9,0,2
2021-09-01,0,0,2,0.001118,1,0,3,1,0.008615,0.011206,9,0,2
2021-09-01,0,0,3,0.000188,0,0,4,2,0.373189,0.453527,9,0,2


### Data Modeling

#### Model 1
Consider using stacked LSTM layers for their ability to represent complex patterns within time series data.

In [25]:
from tensorflow.keras.optimizers import Adam

In [26]:
# define the LSTM model
lst_model = Sequential()

# first LSTM layer with dropout
lst_model.add(LSTM(
    units=1024,
    return_sequences=True,
    activation='swish',
    input_shape=(X_train.shape[1], X_train.shape[2]),
))
lst_model.add(Dropout(0.2))

# second LSTM layer with dropout
lst_model.add(LSTM(
    units=1024,
    return_sequences=True,
    activation='swish'
))
lst_model.add(Dropout(0.2))

# third LSTM layer with dropout, returning only the last output
lst_model.add(LSTM(
    units=1024,
    return_sequences=False,
    activation='swish'
))
lst_model.add(Dropout(0.2))

# dense layer for output
lst_model.add(Dense(units=1))

custom_learning_rate = 0.02
adam_optimizer = Adam(learning_rate=custom_learning_rate)

# compile the model with the custom optimizer
lst_model.compile(optimizer=adam_optimizer, loss='mean_absolute_error')

# compile the model
lst_model.compile(optimizer='adam', loss='mean_absolute_error')

# print the model summary
lst_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 1024)           4239360   
                                                                 
 dropout (Dropout)           (None, 1, 1024)           0         
                                                                 
 lstm_1 (LSTM)               (None, 1, 1024)           8392704   
                                                                 
 dropout_1 (Dropout)         (None, 1, 1024)           0         
                                                                 
 lstm_2 (LSTM)               (None, 1024)              8392704   
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 1)                 1

In [27]:
# set up the EarlyStopping callback to monitor the validation loss
earlyStop = EarlyStopping(
    monitor="val_loss", 
    verbose=1,          # verbose mode will print out extra information
    mode='min',         # the training will stop when the quantity monitored has stopped decreasing
    patience=5          # number of epochs with no improvement after which training will be stopped
)

# fit the LSTM model to the training data
history = lst_model.fit(
    X_train, y_train,                       # training data and labels
    epochs=16,                             # maximum number of epochs to run
    batch_size=1024,                        # batch size for training
    validation_data=(X_test, y_test),       # validation data for evaluating the model
    callbacks=[earlyStop],                  # list of callbacks, in this case just EarlyStopping
    verbose=1,                              # verbose mode will print out extra information per epoch
    shuffle=False                           # don't shuffle the data, usually important in time series
)

Epoch 1/16
1380/1380 [==============================] - 68s 45ms/step - loss: 0.0107 - val_loss: 0.0079
Epoch 2/16
1380/1380 [==============================] - 65s 47ms/step - loss: 0.0072 - val_loss: 0.0062
Epoch 3/16
1380/1380 [==============================] - 65s 47ms/step - loss: 0.0062 - val_loss: 0.0056
Epoch 4/16
1380/1380 [==============================] - 65s 47ms/step - loss: 0.0057 - val_loss: 0.0051
Epoch 5/16
1380/1380 [==============================] - 65s 47ms/step - loss: 0.0054 - val_loss: 0.0049
Epoch 6/16
1380/1380 [==============================] - 66s 47ms/step - loss: 0.0052 - val_loss: 0.0048
Epoch 7/16
1380/1380 [==============================] - 66s 48ms/step - loss: 0.0050 - val_loss: 0.0045
Epoch 8/16
1380/1380 [==============================] - 66s 48ms/step - loss: 0.0049 - val_loss: 0.0044
Epoch 9/16
1380/1380 [==============================] - 66s 48ms/step - loss: 0.0048 - val_loss: 0.0043
Epoch 10/16
1380/1380 [==============================] - 66s 48m

In [28]:
# predict the y_test
y_pred = lst_model.predict(X_test)

18923/18923 [==============================] - 40s 2ms/step


In [29]:
# inverse scaling
y_pred_rescaled = scaler.inverse_transform(y_pred)
y_test_array = y_test.values.reshape(-1, 1)
y_test_rescaled = scaler.inverse_transform(y_test_array.reshape(-1, 1))

# calculate MAE
true_mae = mean_absolute_error(y_test_rescaled, y_pred_rescaled)
print(f"True MAE: {true_mae}")

True MAE: 58.84440003419842


The early stopping callback stopped the training process because the validation loss did not decrease for several epochs, despite potential improvements in training loss. This divergence between training and validation performance is a classic sign of **overfitting**.

#### Model 2
Try reduce the overfitting by increase **dropout** percentage, decrease **stacked layer** number, and adjust learning rate.

In [30]:
# from tensorflow.keras.optimizers import AdamA

In [31]:
# # define the LSTM model
# lst_model_2 = Sequential()

# # first LSTM layer with dropout
# lst_model_2.add(LSTM(
#     units=1024,
#     return_sequences=True,
#     activation='swish',
#     input_shape=(X_train.shape[1], X_train.shape[2]),
# ))
# lst_model_2.add(Dropout(0.3))

# # second LSTM layer with dropout
# lst_model_2.add(LSTM(
#     units=1024,
#     return_sequences=False,
#     activation='swish'
# ))
# lst_model_2.add(Dropout(0.3))

# # dense layer for output
# lst_model_2.add(Dense(units=1))

# # initialize the Adam optimizer with a custom learning rate
# custom_learning_rate = 0.02
# adam_optimizer = Adam(learning_rate=custom_learning_rate)

# # compile the model with the custom optimizer
# lst_model_2.compile(optimizer=adam_optimizer, loss='mean_absolute_error')

# # print the model summary
# lst_model_2.summary()

In [32]:
# # fit the LSTM model to the training data
# history_2 = lst_model_2.fit(
#     X_train, y_train,                       # training data and labels
#     epochs=100,                             # maximum number of epochs to run
#     batch_size=2048,                         # batch size for training
#     validation_data=(X_test, y_test),       # validation data for evaluating the model
#     callbacks=[earlyStop],                  # list of callbacks, in this case just EarlyStopping
#     verbose=2,                              # verbose mode will print out extra information per epoch
#     shuffle=False                           # don't shuffle the data, usually important in time series
# )

In [33]:
# mae = mean_absolute_error(lst_model.predict(test_X), test_y)
# print('Test MAE: %.3f' % mae)

In [34]:
# save model
# lst_model.save('lstm.h5')

### Model 3

Before we tried using 1 timestep for the models, this does not take the advantage of LSTM. Here, we try a 5.

In [35]:
#### read the CSV files into DataFrames
# train = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))

# drop the non-feature columns to create the feature set X
# X = train.drop(['target', 'row_id', 'data_block_id'], axis=1)

# isolate the target variable to create the label set y
# y = train['target']

# print(X.shape)
# print(y.shape)

# X.head(20)


In [36]:
# # convert the DataFrame to a NumPy array if necessary
# X_values = X.values
# y_values = y.values

# # define a function to create sequences from the feature set X
# def create_sequences(X_values, y_values, look_back):
#     X_seq, y_seq = [], []
#     for i in range(len(X_values) - look_back):
#         # retrieve the input sequence
#         sequence = X_values[i:(i + look_back)]
#         X_seq.append(sequence)
#         # retrieve the corresponding target
#         target = y_values[i + look_back]
#         y_seq.append(target)
#     return np.array(X_seq), np.array(y_seq)

# # specify the look_back period
# # try 7!
# look_back = 7  

# # create sequences using the defined function
# X_seq, y_seq = create_sequences(X_values, y_values, look_back)

# print(X_seq.shape)  # This will print the shape of X_seq
# print(y_seq.shape)  # This will print the shape of y_seq

In [37]:
# import tensorflow as tf

# X_seq = tf.convert_to_tensor(X_seq, dtype=tf.float32)
# y_seq = tf.convert_to_tensor(y_seq, dtype=tf.float32)

In [38]:
# # X_seq.shape should be (num_samples, look_back, num_features)
# # y_seq.shape should be (num_samples,)

# # define the LSTM model architecture
# model = Sequential([
#     # LSTM layer with 50 units, input shape is based on the features and look_back
#     LSTM(50, input_shape=(X_seq.shape[1], X_seq.shape[2]), activation='relu'),
#     # Dense layer with one neuron for regression output
#     Dense(1)
# ])

# # compile the model with an optimizer and a loss function
# model.compile(optimizer='adam', loss='mean_squared_error')

# # train the model with the training data
# # use a validation split to monitor the model's performance on unseen data during training
# history = model.fit(
#     X_seq, y_seq,
#     epochs=100,                # Adjust the number of epochs based on your needs
#     batch_size=1024,            # Batch size can be adjusted based on computational resources
#     validation_split=0.2,     # 20% of the data will be used for validation
#     verbose=1                 # Set verbose to 0 for no output, 1 for progress bars, 2 for one line per epoch
# )


## Step 2: Try preprocessed data

Examine the data preprocessing steps detailed here: [Predict Energy Behavior of Prosumers - Data Analysis on Kaggle](https://www.kaggle.com/code/jackren000/predict-energy-behavior-of-prosumers-dataanalysis).

In [39]:
# # load the file
# train = pd.read_csv('/kaggle/input/enefit-processed-train/processed_train.csv')

In [40]:
# train.head(100)